In [ ]:
using Distributions
function generate_data(n, num_obs)
    data = zeros(num_obs, n)

    # Linking factors
    beta = [(i-1.0)/n for i = 1:n] 

    for obs_ind = 1:num_obs
        # Common market factor, mean 3%, sd 2%, truncate at +- 3 sd
        z = rand(Normal(0.03, 0.02))
        z = max(z, 0.03 - 3*0.02)
        z = min(z, 0.03 + 3*0.02)

        for asset_ind = 1:n
            # Idiosyncratic contribution, mean 0%, sd 3%, truncated at +- 3 sd
            asset = rand(Normal(0.00, 0.03))
            asset = max(asset, 0.00 - 3*0.03)
            asset = min(asset, 0.00 + 3*0.03)
            data[obs_ind, asset_ind] = beta[asset_ind] * z + asset
        end
    end
    return 100*data
end
data = generate_data(5,10000);

In [ ]:
data

In [ ]:
using JuMP, Ipopt

n, μ, Σ = 5, vec(mean(data,1)), cov(data)

In [ ]:

r_min = 1.0
port =  Model(solver = IpoptSolver()) 
@variable(port, 0 ≤ x[1:n] ≤ 1)
@constraint(port, sum(x) == 1)
@constraint(port, dot(μ,x) ≥ r_min)
@objective(port, Min, 
    sum(Σ[i,j]*x[i]*x[j] for i=1:n,j=1:n))
solve(port)

In [ ]:
r_range = 1.0:0.1:2.0
rev = zeros(length(r_range ))

for c in 1:length(r_range)
    
    r_min = r_range[c]
    port =  Model(solver = IpoptSolver()) 
    @variable(port, 0 ≤ x[1:n] ≤ 1)
    @constraint(port, sum(x) == 1)
    @constraint(port, dot(μ,x) ≥ r_min)
    @objective(port, Min, 
        sum(Σ[i,j]*x[i]*x[j] for i=1:n,j=1:n))
    solve(port)
    println("solution is $(getobjectivevalue(port))")
    rev[c]=getobjectivevalue(port)
    end    
    

In [ ]:
#Pkg.add("PyPlot")
using PyPlot

#x = linspace(0,2*pi,1000); y = sin.(3*x + 4*cos.(2*x))
#plot(x, y, color="red", linewidth=2.0, linestyle="--")

In [ ]:
#using Gadfly

plot(r_range, rev, color="red", linewidth=2.0, linestyle="--")

In [ ]:
using Gadfly
for r_min in 1.0:0.1:2.0
    port =  Model(solver = IpoptSolver()) 
    @variable(port, 0 ≤ x[1:n] ≤ 1)
    @constraint(port, sum(x) == 1)
    @constraint(port, dot(μ,x) ≥ r_min)
    @objective(port, Min, 
        sum(Σ[i,j]*x[i]*x[j] for i=1:n,j=1:n))
    solve(port)
    plot(x=data*getvalue(x)[:], Geom.density,
         Guide.xlabel("Return"), Guide.ylabel("Density"))
end

In [ ]:





@manipulate for r_min in 1.0:0.1:2.0
    port =  Model(solver = IpoptSolver()) 
    @variable(port, 0 ≤ x[1:n] ≤ 1)
    @constraint(port, sum(x) == 1)
    @constraint(port, dot(μ,x) ≥ r_min)
    @objective(port, Min, 
        sum(Σ[i,j]*x[i]*x[j] for i=1:n,j=1:n))
    solve(port)
    plot(x=data*getvalue(x)[:], Geom.density,
         Guide.xlabel("Return"), Guide.ylabel("Density"))
end



In [ ]:
Pkg.update()

In [ ]:
#Pkg.add("IJulia")
#Pkg.add("Interact")